In [2]:
# IMPORTS
import tensorflow as tf
print( "TensorFlow Version", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

TensorFlow Version 2.7.0


 Load and prpare MNIST dataset

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0 , x_test / 255.0


11501568/11490434 [==============================] - 0s 0us/step


In [7]:
# add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [10]:
# use tf.data to batch and shuffle the dataset
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(1000).batch(32)

In [11]:
test_ds = tf.data.Dataset.from_tensor_slices(
              (x_test, y_test)).batch(32)

Model  Building using keras model subclassing api

In [15]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32,3,activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)
  
  def call(self,x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()

In [16]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [18]:
train_loss = tf.keras.metrics.Mean(name ='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

In [23]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)
  

Test the model

In [28]:
@tf.function
def test_step (images, labels):
  # training = false is only needed if there are layers with 
  # different behavior during training versus inference (e.g Drouput).
  predictions = model(images, training = False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels,predictions)

In [29]:
EPOCHS = 5
for epoch in range(EPOCHS):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)
  
  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)
  
  print(
      f'Epoch {epoch + 1}, '
      f'Loss: {train_loss.result()}, '
      f'Accuracy: {train_accuracy.result() * 100}, '
      f'Test Loss: { test_loss.result()}, '
      f'Test Accuracy: { test_accuracy.result() * 100} '
  )

Epoch 1, Loss: 0.019314223900437355, Accuracy: 99.41999816894531, Test Loss: 0.062828928232193, Test Accuracy: 98.19999694824219 
Epoch 2, Loss: 0.01177261769771576, Accuracy: 99.63333892822266, Test Loss: 0.06634677201509476, Test Accuracy: 98.18000030517578 
Epoch 3, Loss: 0.008178322575986385, Accuracy: 99.7300033569336, Test Loss: 0.06200669705867767, Test Accuracy: 98.54000091552734 
Epoch 4, Loss: 0.006816060747951269, Accuracy: 99.76333618164062, Test Loss: 0.06610159575939178, Test Accuracy: 98.47999572753906 
Epoch 5, Loss: 0.004660174250602722, Accuracy: 99.83666229248047, Test Loss: 0.09619534760713577, Test Accuracy: 97.98999786376953 
